In [27]:
import pandas as pd
import numpy as np
#from datetime import datetime
#dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y')

url = 'https://media.githubusercontent.com/media/reichlab/covid19-forecast-hub/master/data-truth/truth-Incident%20Hospitalizations.csv'
#raw_hosp = pd.read_csv(url, index_col=0, parse_dates=['date'])
raw_hosp = pd.read_csv(url)
print(raw_hosp.head(5))
#x = list(raw_hosp)
#result = np.array(x).astype("float")
#print(result(5))

         date location location_name  value
0  2021-03-02       41        Oregon     19
1  2021-02-24       44  Rhode Island     19
2  2021-02-18       44  Rhode Island     12
3  2021-01-29       23         Maine     15
4  2021-01-29       30       Montana     26


In [30]:
raw_hosp['date'] = pd.to_datetime(raw_hosp.date, infer_datetime_format = True)
#print(raw_hosp.head(5))
raw_hosp.sort_values(by = 'date', ascending = True, inplace = True)
display(raw_hosp.head())

/var/folders/z_/l_q4g1892pqd5g0p2z2f66_m0000gn/T/ipykernel_74198/3617344085.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  raw_hosp['date'] = pd.to_datetime(raw_hosp.date, infer_datetime_format = True)


,date,location,location_name,value
13332,2020-01-01,32,Nevada,132
13619,2020-01-02,32,Nevada,132
14629,2020-01-03,32,Nevada,132
12644,2020-01-03,05,Arkansas,0
13597,2020-01-04,05,Arkansas,0


In [36]:
raw_hosp.date[1]

Timestamp('2021-02-24 00:00:00')

In [45]:
us_hosp = raw_hosp.groupby(["date"]).sum()
us_hosp

,location,location_name,value
date,,,
2020-01-01,32,Nevada,132
2020-01-02,32,Nevada,132
2020-01-03,3205,NevadaArkansas,132
2020-01-04,0532,ArkansasNevada,132
2020-01-05,3205,NevadaArkansas,132
...,...,...,...
2023-04-11,221526110917283747461202US54395536210472012430...,LouisianaHawaiiMichiganDistrict of ColumbiaCon...,4056
2023-04-12,2646134908342028427256604178164021372701253348...,MichiganSouth DakotaGeorgiaUtahColoradoNew Jer...,4112
2023-04-13,3641261024490944603427065121504730041553464254...,New YorkOregonMichiganDelawareMarylandUtahConn...,4128


In [46]:
raw_hosp

,date,location,location_name,value
13332,2020-01-01,32,Nevada,132
13619,2020-01-02,32,Nevada,132
14629,2020-01-03,32,Nevada,132
12644,2020-01-03,05,Arkansas,0
13597,2020-01-04,05,Arkansas,0
...,...,...,...,...
13358,2023-04-15,44,Rhode Island,10
13252,2023-04-15,32,Nevada,14
13220,2023-04-15,50,Vermont,3
74,2023-04-15,78,Virgin Islands,0


In [104]:
us_hosp1 = raw_hosp[raw_hosp["location"] == "US"]
us_hosp1
us_hosp = us_hosp1
length_of_stay = 6
us_hosp["length_of_stay"] = length_of_stay
us_hosp["census"] = 0
us_hosp = us_hosp.reset_index()
us_hosp
mo_hosp = us_hosp.copy()
mo_hosp["value"] = 1

def calculate_census(us_hosp, length_of_stay):
    for i in range(0, len(us_hosp.index) - length_of_stay):
        for j in range(0, length_of_stay + 1):
            us_hosp.census[i + j] += us_hosp.value[i]
            
        us_hosp.census[i + length_of_stay] -= us_hosp.value[i]

    for i in range(len(us_hosp.index) - length_of_stay, len(us_hosp.index)):
        rows_left = len(us_hosp.index) - i
        for j in range(0, rows_left):
            us_hosp.census[i + j] += us_hosp.value[i]
        
    return us_hosp

census = calculate_census(us_hosp, length_of_stay)

/var/folders/z_/l_q4g1892pqd5g0p2z2f66_m0000gn/T/ipykernel_74198/2312434416.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_hosp["length_of_stay"] = length_of_stay
/var/folders/z_/l_q4g1892pqd5g0p2z2f66_m0000gn/T/ipykernel_74198/2312434416.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_hosp["census"] = 0
/var/folders/z_/l_q4g1892pqd5g0p2z2f66_m0000gn/T/ipykernel_74198/2312434416.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

/var/folders/z_/l_q4g1892pqd5g0p2z2f66_m0000gn/T/ipykernel_74198/2312434416.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_hosp.census[i + j] += us_hosp.value[i]
/var/folders/z_/l_q4g1892pqd5g0p2z2f66_m0000gn/T/ipykernel_74198/2312434416.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_hosp.census[i + j] += us_hosp.value[i]
/var/folders/z_/l_q4g1892pqd5g0p2z2f66_m0000gn/T/ipykernel_74198/2312434416.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [105]:
from IPython.display import HTML 
HTML(census.to_html())

,index,date,location,location_name,value,length_of_stay,census
0,54158,2020-07-27,US,United States,6099,6,6099
1,54159,2020-07-28,US,United States,5256,6,11355
2,54160,2020-07-29,US,United States,5387,6,16742
3,54161,2020-07-30,US,United States,6288,6,23030
4,54162,2020-07-31,US,United States,6034,6,29064
5,54163,2020-08-01,US,United States,4504,6,33568
6,54164,2020-08-02,US,United States,4471,6,31940
7,54165,2020-08-03,US,United States,5063,6,31747
8,54166,2020-08-04,US,United States,4844,6,31204
9,54167,2020-08-05,US,United States,4911,6,29827


In [106]:
us_hosp.to_csv('us_hospitalization_census_and_incident_data.csv', columns=["date", "location", "length_of_stay", "value", "census"], 
              header=["date", "location", "length_of_stay", "admissions", "census"])

In [86]:
31940 - 25841

6099

In [67]:
2026 + 8598 - 1761

8863

In [75]:
for i in range(0, length_of_stay):
    print(i)

0
1
2
3
4
5
